
# Setup


## Imports


In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

from functools import partial
import logging
import pathlib
from pathlib import Path
from pprint import pprint
import sys
from typing import *
import yaml
from yaml import YAMLObject

import humanize
from matplotlib import pyplot as plt, cm
import numpy as np
import pandas as pd
from pymicro.file import file_utils
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import utils
from tensorflow.keras import optimizers
from tensorflow.keras import callbacks
from tensorflow.keras import losses

from tomo2seg import modular_unet
from tomo2seg.volume_img_segm import VolumeImgSegmSequence
from tomo2seg.logger import logger
from tomo2seg import data, viz
from tomo2seg.data import Volume, ModelPaths
from tomo2seg.metadata import Metadata

In [3]:
random_state = 42
random_state = np.random.RandomState(random_state)

In [4]:
logger.setLevel(logging.DEBUG)

In [5]:
logger.debug(f"{tf.__version__=}")
logger.info(f"Num GPUs Available: {len(tf.config.list_physical_devices('GPU'))}\nThis should be 2 on R790-TOMO.")
logger.debug(f"Both here should return 2 devices...\n{tf.config.list_physical_devices('GPU')=}\n{tf.config.list_logical_devices('GPU')=}")

# xla auto-clustering optimization (see: https://www.tensorflow.org/xla#auto-clustering)
# this seems to break the training
tf.config.optimizer.set_jit(False)

# get a distribution strategy to use both gpus (see https://www.tensorflow.org/guide/distributed_training)
strategy = tf.distribute.MirroredStrategy()  

DEBUG::tomo2seg::{<ipython-input-5-05ac3d1186aa>:<module>:001}::[2020-11-19::10:26:36.114]
tf.__version__='2.2.0'

INFO::tomo2seg::{<ipython-input-5-05ac3d1186aa>:<module>:002}::[2020-11-19::10:26:36.116]
Num GPUs Available: 0
This should be 2 on R790-TOMO.

DEBUG::tomo2seg::{<ipython-input-5-05ac3d1186aa>:<module>:003}::[2020-11-19::10:26:36.147]
Both here should return 2 devices...
tf.config.list_physical_devices('GPU')=[]
tf.config.list_logical_devices('GPU')=[]

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


# Data

In [6]:
from tomo2seg.datasets import (
#     VOLUME_COMPOSITE_V1 as VOLUME_NAME_VERSION,
#     VOLUME_COMPOSITE_V1_LABELS_REFINED3 as LABELS_VERSION
    VOLUME_PRECIPITATES_DRYRUN_VAL as VOLUME_NAME_VERSION
)
LABELS_VERSION = None

volume_name, volume_version = VOLUME_NAME_VERSION
labels_version = LABELS_VERSION

logger.info(f"{volume_name=} {volume_version=} {labels_version=}")

INFO::tomo2seg::{<ipython-input-6-21e9ac780e91>:<module>:011}::[2020-11-19::10:26:36.202]
volume_name='PA66GF30_trans3_x__0_pag' volume_version='val' labels_version=None



In [13]:
# Metadata/paths objects

## Volume
volume = Volume.with_check(
    name=volume_name, version=volume_version
)
logger.info(f"{volume=}")

def _read_raw(path_: Path, volume_: Volume): 
    # from pymicro
    return file_utils.HST_read(
        str(path_),  # it doesn't accept paths...
        # pre-loaded kwargs
        autoparse_filename=False,  # the file names are not properly formatted
        data_type=volume.metadata.dtype,
        dims=volume.metadata.dimensions,
        verbose=True,
    )

read_raw = partial(_read_raw, volume_=volume)

logger.info("Loading data from disk.")

## Data
voldata = read_raw(volume.data_path)
logger.debug(f"{voldata.shape=}")

voldata_train = volume.train_partition.get_volume_partition(voldata)
voldata_val = volume.val_partition.get_volume_partition(voldata)
logger.debug(f"{voldata_train.shape=} {voldata_val.shape=}")

del voldata

## Labels
vollabels = read_raw(volume.versioned_labels_path(labels_version))
logger.debug(f"{vollabels.shape=}")

vollabels_train = volume.train_partition.get_volume_partition(vollabels)
vollabels_val = volume.val_partition.get_volume_partition(vollabels)
logger.debug(f"{vollabels_train.shape=} {vollabels_val.shape=}")

del vollabels

DEBUG::tomo2seg::{data.py:with_check:218}::[2020-11-19::10:30:46.447]
vol=Volume(name='PA66GF30_trans3_x__0_pag', version='val', _metadata=None)

Missing file: /home/joaopcbertoldo/projects/tomo2seg/data/PA66GF30_trans3_x__0_pag.val/PA66GF30_trans3_x__0_pag.val.weights.raw

DEBUG::tomo2seg::{data.py:metadata:175}::[2020-11-19::10:30:46.451]
Loading metadata from `/home/joaopcbertoldo/projects/tomo2seg/data/PA66GF30_trans3_x__0_pag.val/PA66GF30_trans3_x__0_pag.val.metadata.yml`.

INFO::tomo2seg::{<ipython-input-13-d495b0b61438>:<module>:007}::[2020-11-19::10:30:46.462]
volume=Volume(name='PA66GF30_trans3_x__0_pag', version='val', _metadata=Volume.Metadata(dimensions=[500, 500, 500], dtype='uint8', labels=[0, 1, 2], labels_names={0: 'matrix', 1: 'fiber', 2: 'porosity'}, set_partitions={'train': {'x_range': [0, 500], 'y_range': [0, 500], 'z_range': [0, 200], 'alias': 'train'}, 'val': {'x_range': [0, 500], 'y_range': [0, 500], 'z_range': [200, 300], 'alias': 'val'}, 'test': {'x_range': [0,

## Data Sequences

In [24]:
from tomo2seg.volume_sequence import VolumeCropSequence, MetaCrop3DGenerator, ET3DUniformCuboidAlmostEverywhere, UniformGridPosition, UniformGTEverywhere

In [21]:
batch_size_per_replica = 16  
batch_size = batch_size_per_replica * (n_replicas := strategy.num_replicas_in_sync)

# crop_size = 224  # multiple of 16 (requirement of a 4-level u-net)
logger.info(f"{batch_size_per_replica=}\n{n_replicas=}\n{batch_size=}")

INFO::tomo2seg::{<ipython-input-21-3265efa5042f>:<module>:005}::[2020-11-19::10:44:32.867]
batch_size_per_replica=16
n_replicas=1
batch_size=16



In [ ]:
volume_shape = voldata_train.shape
crop_shape = (224, 224, 1)

crop_xyz_ranges = {
    "x_range": (0, volume_shape[0] - crop_shape[0]),
    "y_range": (0, volume_shape[1] - crop_shape[1]),
    "z_range": (0, volume_shape[2] - crop_shape[2]),
}

crop_seq_train = VolumeCropSequence(
    data_volume=voldata_train,
    labels_volume=vollabels_train,
    labels=volume.metadata.labels,
    meta_crop_generator=MetaCrop3DGenerator(
        volume_shape=volume_shape,
        crop_shape=crop_shape,
        x0y0z0_generator=UniformGridPosition(
            
        )
    ),
    batch_size=batch_size,
    # this volume cropper only returns random crops, 
    #so the number of crops per epoch/batch is w/e i want
    epoch_size=100,
    meta_crops_hist_path=None,  # todo add a new path to the model and save this
)

In [25]:
?UniformGridPosition

Init signature:
UniformGridPosition(
    x_range: Tuple[int, int],
    y_range: Tuple[int, int],
    z_range: Tuple[int, int],
    random_state: numpy.random.mtrand.RandomState,
) -> None
Docstring:      RandomGridPositionGenerator(x_range: Tuple[int, int], y_range: Tuple[int, int], z_range: Tuple[int, int], random_state: numpy.random.mtrand.RandomState)
File:           ~/projects/tomo2seg/tomo2seg/volume_sequence.py
Type:           ABCMeta
Subclasses:     


In [23]:
?MetaCrop3DGenerator

Init signature:
MetaCrop3DGenerator(
    volume_shape: Tuple[int, int, int],
    crop_shape: Tuple[int, int, int],
    x0y0z0_generator: tomo2seg.volume_sequence.RandomGridPositionGenerator,
    elastic_transformation_field: tomo2seg.volume_sequence.ProbabilityField3D,
    geometric_transformation_field: tomo2seg.volume_sequence.ProbabilityField3D,
    value_shift_field: tomo2seg.volume_sequence.ProbabilityField3D,
) -> None
Docstring:      MetaCrop3DGenerator(volume_shape: Tuple[int, int, int], crop_shape: Tuple[int, int, int], x0y0z0_generator: tomo2seg.volume_sequence.RandomGridPositionGenerator, elastic_transformation_field: tomo2seg.volume_sequence.ProbabilityField3D, geometric_transformation_field: tomo2seg.volume_sequence.ProbabilityField3D, value_shift_field: tomo2seg.volume_sequence.ProbabilityField3D)
File:           ~/projects/tomo2seg/tomo2seg/volume_sequence.py
Type:           type
Subclasses:     


In [ ]:
batch_size_per_replica = 16  
batch_size = batch_size_per_replica * mirrored_strategy.num_replicas_in_sync
crop_size = 224  # multiple of 16 (requirement of a 4-level u-net)
n_geometric_augmentations = 3
slice_axes = (0, 1, 2,)

train_generator = VolumeImgSegmSequence(
    source_volume=train_volume, 
    label_volume=train_labels,
    labels=labels_list, 
    axes=slice_axes, 
    batch_size=batch_size,
    shuffle=True,
    normalization_const=255,
    n_geometric_augmentations=n_geometric_augmentations,
    random_state=42,
    crop_size=crop_size,
)

val_generator = VolumeImgSegmSequence(
    source_volume=val_volume, 
    label_volume=val_labels,
    labels=labels_list, 
    axes=(2,), 
    batch_size=10,
    shuffle=False,
    normalization_const=255,
    n_geometric_augmentations=0,
    crop_size=crop_size,
)

# Model

In [ ]:
model_name = "dryrun-03"
model_paths = ModelPaths(model_name)
logger.info("Model paths object: %s", model_paths)

input_shape = (crop_size, crop_size, 1)
nb_filters_0 = 12
model_generator_function = modular_unet.u_net

# lr = 4e-3
# n_epochs = 30


In [ ]:
with mirrored_strategy.scope():
    if not pathlib.Path(str(model_paths.autosaved_model_path) + '.hdf5').exists():
        assert not model_paths.model_path.exists(), f"Please delete '{model_paths.model_path}' to redefine it."

        model = model_generator_function(input_shape, nb_filters_0=nb_filters_0, output_channels=3, name=model_name)
        model.save(filepath=model_paths.model_path, overwrite=False)

    else:
        logger.warning("An autosaved model already exists, loading it instead of creating a new one!")
        model = keras.models.load_model(str(model_paths.autosaved_model_path) + '.hdf5')


#     optimizer = optimizers.Adam(lr=lr)
    optimizer = optimizers.Adam()
    loss_func = losses.categorical_crossentropy    
    
    model.compile(loss=loss_func, optimizer=optimizer)
    model.save(model_paths.model_path)

In [ ]:
# write the model summary in a file
with model_paths.summary_path.open("w") as f:
    def print_to_txt(line):
        f.writelines([line + "\n"])
    model.summary(print_fn=print_to_txt, line_length=140)

# same for the architecture
utils.plot_model(model, show_shapes=True, to_file=model_paths.architecture_plot_path);



# Callbacks

In [ ]:

autosave_cb = callbacks.ModelCheckpoint(
    str(model_paths.autosaved_model_path) + ".hdf5", 
    monitor="val_loss", 
    verbose=0, 
    save_best_only=True, 
    mode="auto",
)

reduce_lr_cb = ReduceLROnPlateauBacktrack(
    model,
    str(model_paths.autosaved_model_path) + ".hdf5", 
    monitor="val_loss",
    factor=0.5,
    patience=5,
    verbose=1,
    mode="auto",
    min_delta=0,
    min_lr=1e-5,
),

logger_cb = callbacks.CSVLogger(
    str(model_paths.logger_path), separator=",", append=False
)

def get_logspace_lr_cb(min_lr, max_lr, n_epochs, epoch_zero=0):
    schedule = np.logspace(min_lr, max_lr, n_epochs)
    def schedule_function(epoch, lr):
        assert epoch - epoch_zero < n_epochs, "Schedule is over!"
        return schedule[epoch - epoch_zero]
    return tf.keras.callbacks.LearningRateScheduler(schedule_function)

lr_range_test_schedule_cb = get_logspace_lr_cb(-4.5, -1, 30)

In [ ]:
# unique, counts = np.unique(train_labels, return_counts=True)
# class_freqs = dict(zip(unique, counts))
# total = sum(class_freqs.values())
# class_freqs = {k: v / total for k, v in class_freqs.items()}
# class_freqs
# class_freqs_inv = {k: 1. / v for k, v in class_freqs.items()}
# class_freqs_inv
# import tensorflow as tf
# from tensorflow.python.keras import backend as K
# weights = [class_freqs_inv[i] for i in range(len(class_freqs_inv))]

# def weighted_cross_entropy(y_true, y_pred):
#   Kweights = K.constant(weights)
#   if not K.is_keras_tensor(y_pred):
#     y_pred = K.constant(y_pred)
#   y_true = K.cast(y_true, y_pred.dtype)
#   return K.categorical_crossentropy(y_true, y_pred) * K.sum(y_true * y_pred, axis=-1)

# Summary before training

In [ ]:
# stuff that i use after the training but i want it to appear in the 
# pre-training summary

## Metadata

## Volume slices

## Generator samples

# Learning rate range test

In [ ]:
cb = [
    autosave_cb,
    logger_cb,
    callbacks.TerminateOnNaN(),
#     reduce_lr_cb,
    lr_range_test_schedule_cb
]

train_generator.force_shorter_epoch = 100

In [ ]:
history_lr_rate_range_test = model.fit(
    x=train_generator,
    validation_data=val_generator,
    validation_steps=100,  
    epochs=30,  
    callbacks=cb,  
    verbose=1,
    use_multiprocessing=False,   
    
)

In [ ]:
plt.subplots(figsize=(10,10))
plt.tight_layout()
viz.display_training_curves(
    history_lr_rate_range_test.history['loss'], history_lr_rate_range_test.history['val_loss'], 'loss', 111, x=np.logspace(-4.5, -1, 30)
)
plt.xscale('log')

# Training


In [ ]:
model = tf.keras.models.load_model(str(model_paths.autosaved_model_path) + ".hdf5")

In [ ]:
lr = 0.001

from keras import backend as K
K.set_value(model.optimizer.learning_rate, lr)

cb = [
    autosave_cb,
    logger_cb,
    callbacks.TerminateOnNaN(),
    reduce_lr_cb,
#     lr_range_test_schedule_cb
]

train_generator.force_shorter_epoch = 100

In [ ]:
history = model.fit(
    x=train_generator,
    validation_data=val_generator,
    validation_steps=100,  
    epochs=40,  
    callbacks=cb,  
    verbose=1,
    use_multiprocessing=False,   
    
    initial_epoch=30,
)

In [ ]:
plt.subplots(figsize=(10,10))
plt.tight_layout()
viz.display_training_curves(history.history['loss'], history.history['val_loss'], 'loss', 111)

# Training 2nd round

In [ ]:
model.optimizer.learning_rate / 2

In [ ]:
reduce_lr_cb = ReduceLROnPlateauBacktrack(
    model,
    str(model_paths.autosaved_model_path) + ".hdf5",  
    monitor="val_loss",
    factor=0.5,
    patience=4,
    verbose=1,
    mode="auto",
    min_delta=0,
    min_lr=1e-5,
)

lr = 1.5773934e-05

from tensorflow.keras import backend as K
K.set_value(model.optimizer.learning_rate, lr)

cb = [
    autosave_cb,
    logger_cb,
    callbacks.TerminateOnNaN(),
    reduce_lr_cb,
#     lr_range_test_schedule_cb
]

train_generator.force_shorter_epoch = 300

In [ ]:
history_2 = model.fit(
    x=train_generator,
    validation_data=val_generator,
    validation_steps=100,  
    epochs=55,  
    callbacks=cb,  
    verbose=1,
    use_multiprocessing=False,   
    
    initial_epoch=40,
)

In [ ]:
with mirrored_strategy.scope():
    model = tf.keras.models.load_model(str(model_paths.autosaved_model_path) + ".hdf5")

In [ ]:
reduce_lr_cb = ReduceLROnPlateauBacktrack(
    model,
    str(model_paths.autosaved_model_path) + ".hdf5",  
    monitor="val_loss",
    factor=2./3,
    patience=3,
    verbose=1,
    mode="auto",
    min_delta=0,
    min_lr=1e-7,
)

lr = 1e-05

from tensorflow.keras import backend as K
K.set_value(model.optimizer.learning_rate, lr)

cb = [
    autosave_cb,
    logger_cb,
    callbacks.TerminateOnNaN(),
    reduce_lr_cb,
#     lr_range_test_schedule_cb
]

train_generator.force_shorter_epoch = None

In [ ]:
model.save(model_paths.model_path)